In [1]:
import cfscrape
from lxml import etree
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from datetime import date
from itertools import cycle
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'}

## Getting links

In [237]:
section_links = ['https://www.holzland.de/holzplatten/?p={}',
                'https://www.holzland.de/massivholz/?p={}',
                'https://www.holzland.de/paneele/?p={}',
                'https://www.holzland.de/terrassendielen/?p={}']
pages_num = [10, 5, 10, 10]

In [243]:
sections_links = []
for i, section_link in enumerate(section_links):
    print('Section', i)
    
    for page in range(pages_num[i]+1):
        url = section_links[i].format(page)
        browser.get(url)
        sleep(2)
        path = '//*[@class="row prd-grid tiles"]//a'
        page_links = browser.find_elements_by_xpath(path)
        
        for url in page_links:
            link = url.get_attribute('href')
            if link not in sections_links and 'select-merchant' not in link:
                sections_links.append(link)
            
    print('All Links:', len(sections_links))

Section 0
All Links: 600
Section 1
All Links: 861
Section 2
All Links: 1461
Section 3
All Links: 2060


In [545]:
import pickle
with open('HolzLandLInks.pkl', 'wb') as f:
    pickle.dump(sections_links, f)

### Testing Options

In [574]:
%%time
options_df = pd.DataFrame(columns=['URL','Options'], index=range(len(sections_links)))
for idx in options_df.index[:]:
    sleep(randint(1,2))
    url = sections_links[idx]
    browser.get(url)
    
    path = '//*[@class="form-group"]'
    options = []
    for el in browser.find_elements_by_xpath(path):
        opt = el.text
        opt = opt.split('\n')[0]
        if len(opt)>0 and 'MENGE' not in opt:
            options.append(opt)
    
    options_df["URL"].loc[idx] = url
    options_df["Options"].loc[idx] = ', '.join(options)
    
    if idx%100==0:
        print(idx, url, ', '.join(options), options_df.isna().sum())

0 https://www.holzland.de/p/osb3-verlegeplatte-e1-ungeschliffen/HL523253/?pmzr=12013&itemId=7000849625 LÄNGE, BREITE, STÄRKE URL        2059
Options    2059
dtype: int64
100 https://www.holzland.de/p/topan-mdf-formbar-e1/HL102070/?pmzr=10082&itemId=7000002571  URL        1959
Options    1959
dtype: int64
200 https://www.holzland.de/p/kuechenarbeitsplatte-af-40133-hydro-einseitig-gerundet-bz173-si-bronzit-schwarz/HL274095/?pmzr=99996&itemId=7000315195 BREITE URL        1859
Options    1859
dtype: int64
300 https://www.holzland.de/p/kuechenarbeitsplatte-af-40133-hydro-beids-gerundet-bz173-si-bronzit-schwarz/HL274423/?pmzr=99996&itemId=7000315741 BREITE URL        1759
Options    1759
dtype: int64
400 https://www.holzland.de/p/schichtstoff-hpl-hgp-a1-pe-schwarz/HL271286/?pmzr=99996&itemId=7000312386  URL        1659
Options    1659
dtype: int64
500 https://www.holzland.de/p/kuechenarbeitsplatte-af-30133-ds-design-einseitig-gerundet-l750-t-twist/HL272532/?pmzr=99996&itemId=7000313632 BREIT

In [580]:
options_df.to_csv('Holzland_Options_URLs.csv')

In [553]:
options_df[(options_df['Options']!='LÄNGE, BREITE, STÄRKE') &
          (options_df['Options'].isin(options))].shape

(217, 3)

In [582]:
options_df['Options'].value_counts()

                                    865
LÄNGE                               596
BREITE                              236
LÄNGE, BREITE                       178
LÄNGE, BREITE, STÄRKE                95
STÄRKE                               45
LÄNGE, STÄRKE                        20
BREITE, STÄRKE                       10
LÄNGE, DIMENSION                     10
DIMENSION                             3
LÄNGE, DURCHMESSER                    1
LÄNGE, BREITE, STÄRKE, DIMENSION      1
Name: Options, dtype: int64

## Testing scraping page

In [563]:
browser.quit()

In [610]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless=False
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)
url = 'https://www.holzland.de/p/fasebrett-nordische-fichte-us-hobelfallend/HL571200/?pmzr=12013&itemId=7000612227'
url = 'https://www.holzland.de/p/konstruktionsholz-kieferfichte-kdi-gruen-gehobelt-gefast-staerke-ab-40-mm/HL1471671/?pmzr=10089&itemId=7001382347'
browser.get(url)

In [558]:
options_names = ['LÄNGE, BREITE', 'LÄNGE, STÄRKE', 'LÄNGE, DIMENSION','BREITE, STÄRKE',
                   'LÄNGE, BREITE, STÄRKE, DIMENSION']

## Option less than three dimentions

In [567]:
%%time
#df = pd.DataFrame()
for p, idx in enumerate(options_df[(options_df['Options']!='LÄNGE, BREITE, STÄRKE') &
                                   (options_df['Options'].isin(options_names))].index):
    url = options_df['URL'].loc[idx]
    browser.get(url)
    
    print('Product', p)
    
    # Metadata for choosing Options
    path = '//*[@name="var[import:Laenge_Artikel]"]//option'
    l_options = browser.find_elements_by_xpath(path)

    path = '//*[@name="var[import:Breite_Artikel]"]//option'
    w_options = browser.find_elements_by_xpath(path)

    path = '//*[@name="var[import:Staerke_Artikel]"]//option'
    h_options = browser.find_elements_by_xpath(path)
    
    path = '//*[@name="var[import:Dimension]"]//option'
    d_options = browser.find_elements_by_xpath(path)
    
    l,w,h,d = len(l_options), len(w_options), len(h_options), len(d_options)
    
    sleep_time = 1
    product_df = pd.DataFrame()

    if l>0:
        first_option = l
    else:
        first_option = w

    for i in range(first_option): 
        first_idx = i+1

        # Length
        if l>0:
            path = '//*[@name="var[import:Laenge_Artikel]"]//option[{}]'.format(first_idx)
            exp_l =  browser.find_elements_by_xpath(path)[0].text
            exp_l = re.sub(r'[^\.\d]','', exp_l)
            browser.find_elements_by_xpath(path)[0].click()
            sleep(sleep_time)

        else:
            path = '//*[@name="var[import:Breite_Artikel]"]//option[{}]'.format(first_idx)
            exp_l =  browser.find_elements_by_xpath(path)[0].text
            exp_l = re.sub(r'[^\.\d]','', exp_l)
            browser.find_elements_by_xpath(path)[0].click()
            sleep(sleep_time)


        if d>0:
            second_option = 'Dimension'
            check='CHECK'

        elif w>0 and d==0 and h==0:
            second_option = 'Breite_Artikel'
            check = 'Ihre Konfiguration ändert sich'
        elif h>0 and d==0:
            second_option = 'Staerke_Artikel'
            check = 'Ihre Konfiguration ändert sich'

        path = '//*[@name="var[import:{}]"]//option'.format(second_option)
        options = browser.find_elements_by_xpath(path)
        right_opt = [options.index(opt) for opt in options if check not in opt.text]

        for idx in right_opt:
            path = '//*[@name="var[import:{}]"]/option[{}]'.format(second_option, idx+1)
            exp_w =  browser.find_elements_by_xpath(path)[0].text
            exp_w = re.sub(r'[^\.\d]','', exp_w)
            browser.find_elements_by_xpath(path)[0].click()
            sleep(1)

            # Getting Dimentions
            path = '//*[@class="prd-details text--small"]'
            raw_dims = browser.find_elements_by_xpath(path)[0].text

            # Price
            try:
                path = '//*[@class="delivery-price-with-unit"]'
                price = browser.find_elements_by_xpath(path)[0].text
            except:
                path = '//*[@class="sum"]'
                price = browser.find_elements_by_xpath(path)[0].text

            #SKU
            path = '//*[@class="article-number text--small color-gray-light"]'
            sku = browser.find_elements_by_xpath(path)[0].text

            # Title
            path = '//*[@class="h3 headline"]'
            title = browser.find_elements_by_xpath(path)[0].text
            
            # Getting Main Data
            path = '//*[@class="prd-features__feature-list js-prd-features__feature-list prd-features__feature-list--visible"]//dd'
            values = [el.text for el in browser.find_elements_by_xpath(path)]

            path = '//*[@class="prd-features__feature-list js-prd-features__feature-list prd-features__feature-list--visible"]//dt'
            cols = [el.text for el in browser.find_elements_by_xpath(path)]
            main_data = pd.DataFrame([values], columns=cols)

            item_df = pd.DataFrame({'SKU':sku,
                                    'Title':title,
                                    'Raw Dims':raw_dims,
                                    'Price':price,
                                    'URL':browser.current_url}, index=[0])
            
            item_df = pd.concat([item_df, main_data], axis=1)
            product_df = pd.concat([product_df,item_df])
            sleep(1)


    df = pd.concat([df, product_df], ignore_index=True)
    df.to_csv('Holzland_Options.csv')

    print('Product size:', product_df.shape[0], '|', 'All products size:', df.shape[0])


Product 0
Product size: 16 | All products size: 16
Product 1
Product size: 9 | All products size: 25
Product 2
Product size: 13 | All products size: 38
Product 3
Product size: 9 | All products size: 47
Product 4
Product size: 9 | All products size: 56
Product 5
Product size: 9 | All products size: 65
Product 6
Product size: 9 | All products size: 74
Product 7
Product size: 3 | All products size: 77
Product 8
Product size: 9 | All products size: 86
Product 9
Product size: 6 | All products size: 92
Product 10
Product size: 7 | All products size: 99
Product 11
Product size: 9 | All products size: 108
Product 12
Product size: 9 | All products size: 117
Product 13
Product size: 9 | All products size: 126
Product 14
Product size: 9 | All products size: 135
Product 15
Product size: 9 | All products size: 144
Product 16
Product size: 9 | All products size: 153
Product 17
Product size: 9 | All products size: 162
Product 18
Product size: 9 | All products size: 171
Product 19
Product size: 9 | Al

Product 157
Product size: 666 | All products size: 3857
Product 158
Product size: 108 | All products size: 3965
Product 159
Product size: 14 | All products size: 3979
Product 160
Product size: 16 | All products size: 3995
Product 161
Product size: 8 | All products size: 4003
Product 162
Product size: 63 | All products size: 4066
Product 163
Product size: 10 | All products size: 4076
Product 164
Product size: 8 | All products size: 4084
Product 165
Product size: 36 | All products size: 4120
Product 166
Product size: 40 | All products size: 4160
Product 167
Product size: 24 | All products size: 4184
Product 168
Product size: 24 | All products size: 4208
Product 169
Product size: 40 | All products size: 4248
Product 170
Product size: 7 | All products size: 4255
Product 171
Product size: 42 | All products size: 4297
Product 172
Product size: 18 | All products size: 4315
Product 173
Product size: 4 | All products size: 4319
Product 174
Product size: 20 | All products size: 4339
Product 175


In [585]:
df[~df.duplicated(['SKU','Price'])].shape

(1716, 39)

## One Dimention

In [586]:
one_dim_names = ['BREITE','LÄNGE','STÄRKE','DIMENSION']

In [588]:
options_df[options_df['Options'].isin(one_dim_names)].shape

(880, 2)

In [613]:
%%time
df_one_dim = pd.DataFrame()
for p, idx in enumerate(options_df[options_df['Options'].isin(one_dim_names)].index):
    url = options_df['URL'].loc[idx]
    browser.get(url)
    
    print('Product', p)
    
    # Metadata for choosing Options
    path = '//*[@name="var[import:Laenge_Artikel]"]//option'
    l_options = browser.find_elements_by_xpath(path)

    path = '//*[@name="var[import:Breite_Artikel]"]//option'
    w_options = browser.find_elements_by_xpath(path)

    path = '//*[@name="var[import:Staerke_Artikel]"]//option'
    h_options = browser.find_elements_by_xpath(path)
    
    path = '//*[@name="var[import:Dimension]"]//option'
    d_options = browser.find_elements_by_xpath(path)
    
    l,w,h,d = len(l_options), len(w_options), len(h_options), len(d_options)
    
    sleep_time = 1
    product_df = pd.DataFrame()

    if l>0:
        first_option = l
        first_option_name = 'Laenge_Artikel'
    
    elif w>0:
        first_option = w
        first_option_name = 'Breite_Artikel'
    
    elif h>0:
        first_option = h
        first_option_name = 'Staerke_Artikel'
        
    elif d>0:
        first_option = d
        first_option_name = 'Dimension'

    
    for i in range(first_option): 
        first_idx = i+1

        # First Opt
        
        path = '//*[@name="var[import:{}]"]//option[{}]'.format(first_option_name, first_idx)
        exp_l =  browser.find_elements_by_xpath(path)[0].text
        exp_l = re.sub(r'[^\.\d]','', exp_l)
        browser.find_elements_by_xpath(path)[0].click()
        sleep(sleep_time)

        # Getting Dimentions
        path = '//*[@class="prd-details text--small"]'
        raw_dims = browser.find_elements_by_xpath(path)[0].text

        # Price
        try:
            path = '//*[@class="delivery-price-with-unit"]'
            price = browser.find_elements_by_xpath(path)[0].text
        except:
            path = '//*[@class="sum"]'
            price = browser.find_elements_by_xpath(path)[0].text

        #SKU
        path = '//*[@class="article-number text--small color-gray-light"]'
        sku = browser.find_elements_by_xpath(path)[0].text

        # Title
        path = '//*[@class="h3 headline"]'
        title = browser.find_elements_by_xpath(path)[0].text

        # Try to show more data
        try:
            path = '//*[@class="icon nm-icon nm-icon-arrow-down"]'
            browser.find_elements_by_xpath(path)[0].click()
        except:
            pass
        
        # Getting Main Data
        path = '//*[@class="prd-features__feature-list js-prd-features__feature-list prd-features__feature-list--visible"]//dd'
        values = [el.text for el in browser.find_elements_by_xpath(path)]

        path = '//*[@class="prd-features__feature-list js-prd-features__feature-list prd-features__feature-list--visible"]//dt'
        cols = [el.text for el in browser.find_elements_by_xpath(path)]
        main_data = pd.DataFrame([values], columns=cols)

        item_df = pd.DataFrame({'SKU':sku,
                                'Title':title,
                                'Raw Dims':raw_dims,
                                'Price':price,
                                'URL':browser.current_url}, index=[0])
        
        item_df = pd.concat([item_df, main_data], axis=1)
        product_df = pd.concat([product_df,item_df])
        sleep(1)
        
        if p%20==0:
            display(item_df)


    df_one_dim = pd.concat([df_one_dim, product_df], ignore_index=True)
    #df.to_csv('Holzland_Options.csv')

    print('Product size:', product_df.shape[0], '|', 'All products size:', df_one_dim.shape[0])



Product 0


,SKU,Title,Raw Dims,Price,URL,Verleimung,Länge,Breite,Stärke
0,Artikelnummer: 7001156775,"Betonschalungsplatte Pappel d+d, WBP, film/film",2500 x 1250 x 4 mm,"28,00 € / Stk.",https://www.holzland.de/p/betonschalungsplatte...,WBP,2500 mm,1250 mm,4 mm


,SKU,Title,Raw Dims,Price,URL,Verleimung,Länge,Breite,Stärke
0,Artikelnummer: 7001156778,"Betonschalungsplatte Pappel d+d, WBP, film/film",2500 x 1250 x 8 mm,"35,03 € / Stk.",https://www.holzland.de/p/betonschalungsplatte...,WBP,2500 mm,1250 mm,8 mm


,SKU,Title,Raw Dims,Price,URL,Verleimung,Länge,Breite,Stärke
0,Artikelnummer: 7001156776,"Betonschalungsplatte Pappel d+d, WBP, film/film",2500 x 1250 x 20 mm,"57,56 € / Stk.",https://www.holzland.de/p/betonschalungsplatte...,WBP,2500 mm,1250 mm,20 mm


Product size: 3 | All products size: 3
Product 1
Product size: 10 | All products size: 13
Product 2
Product size: 3 | All products size: 16
Product 3
Product size: 4 | All products size: 20
Product 4
Product size: 7 | All products size: 27
Product 5
Product size: 7 | All products size: 34
Product 6
Product size: 9 | All products size: 43
Product 7
Product size: 10 | All products size: 53
Product 8
Product size: 6 | All products size: 59
Product 9
Product size: 12 | All products size: 71
Product 10
Product size: 11 | All products size: 82
Product 11
Product size: 2 | All products size: 84
Product 12
Product size: 6 | All products size: 90
Product 13
Product size: 7 | All products size: 97
Product 14
Product size: 5 | All products size: 102
Product 15
Product size: 2 | All products size: 104
Product 16
Product size: 5 | All products size: 109
Product 17
Product size: 5 | All products size: 114
Product 18
Product size: 5 | All products size: 119
Product 19
Product size: 8 | All products s

...

,SKU,Title,Raw Dims,Price,URL,Aufbau,Länge,Marke,Breite,Materialdetailtyp,Stärke,Oberflächenstruktur Oben/Unten,Biegefestigkeit,pflegeleicht,Farbton,Farbe
0,Artikelnummer: 7000456706,Barth Terrassendiele BPC Hohlkammer geschliffe...,Länge 500 cm,"39,95 € / Stk.",https://www.holzland.de/p/terrassendiele-bpc-h...,Hohlkammer,5000 mm,Barth,145 mm,BPC (Bamboo Plastic Composite),25 mm,"1-seitig geschliffen, 1-seitig gebürstet","20,68 N/mm²",ja,hell,silver cedar (grau)


Product size: 3 | All products size: 2998
Product 841
Product size: 7 | All products size: 3005
Product 842
Product size: 5 | All products size: 3010
Product 843
Product size: 3 | All products size: 3013
Product 844
Product size: 3 | All products size: 3016
Product 845
Product size: 2 | All products size: 3018
Product 846
Product size: 3 | All products size: 3021
Product 847
Product size: 3 | All products size: 3024
Product 848
Product size: 7 | All products size: 3031
Product 849
Product size: 3 | All products size: 3034
Product 850
Product size: 4 | All products size: 3038
Product 851
Product size: 3 | All products size: 3041
Product 852
Product size: 6 | All products size: 3047
Product 853
Product size: 7 | All products size: 3054
Product 854
Product size: 3 | All products size: 3057
Product 855
Product size: 2 | All products size: 3059
Product 856
Product size: 3 | All products size: 3062
Product 857
Product size: 4 | All products size: 3066
Product 858
Product size: 7 | All produc

,SKU,Title,Raw Dims,Price,URL,Aufbau,Länge,Marke,Breite,Materialdetailtyp,Stärke,Oberflächenstruktur Oben/Unten,Farbe
0,Artikelnummer: 7001479751,UPM Terrassendiele WPC massiv geprägt Streaked...,Länge 500 cm,"73,95 € / Stk.",https://www.holzland.de/p/terrassendiele-wpc-m...,massiv,5000 mm,UPM,140 mm,WPC (Wood Plastic Composite),25 mm,geprägt,Streaked Ebony


,SKU,Title,Raw Dims,Price,URL,Aufbau,Länge,Marke,Breite,Materialdetailtyp,Stärke,Oberflächenstruktur Oben/Unten,Farbe
0,Artikelnummer: 7001479751,UPM Terrassendiele WPC massiv geprägt Streaked...,Länge 500 cm,"73,95 € / Stk.",https://www.holzland.de/p/terrassendiele-wpc-m...,massiv,5000 mm,UPM,140 mm,WPC (Wood Plastic Composite),25 mm,geprägt,Streaked Ebony


Product size: 2 | All products size: 3078
Product 861
Product size: 3 | All products size: 3081
Product 862
Product size: 3 | All products size: 3084
Product 863
Product size: 5 | All products size: 3089
Product 864
Product size: 3 | All products size: 3092
Product 865
Product size: 5 | All products size: 3097
Product 866
Product size: 3 | All products size: 3100
Product 867
Product size: 2 | All products size: 3102
Product 868
Product size: 2 | All products size: 3104
Product 869
Product size: 2 | All products size: 3106
Product 870
Product size: 2 | All products size: 3108
Product 871
Product size: 2 | All products size: 3110
Product 872
Product size: 2 | All products size: 3112
Product 873
Product size: 2 | All products size: 3114
Product 874
Product size: 2 | All products size: 3116
Product 875
Product size: 2 | All products size: 3118
Product 876
Product size: 2 | All products size: 3120
Product 877
Product size: 2 | All products size: 3122
Product 878
Product size: 2 | All produc

In [477]:
product_df.shape

(4, 4)

In [616]:
df_one_dim.to_csv('Holzland_1D-Options.csv')

## No Options

In [619]:
options_df[options_df['Options']==''].shape

(865, 2)

In [621]:
%%time
df_no_opt = pd.DataFrame()
for p, idx in enumerate(options_df[options_df['Options']==''].index):
    url = options_df['URL'].loc[idx]
    browser.get(url)
    
    print('Product', p)

    sleep_time = 1
    product_df = pd.DataFrame()

   
    # Getting Dimentions
    path = '//*[@class="prd-details text--small"]'
    raw_dims = browser.find_elements_by_xpath(path)[0].text

    # Price
    try:
        path = '//*[@class="delivery-price-with-unit"]'
        price = browser.find_elements_by_xpath(path)[0].text
    except:
        path = '//*[@class="sum"]'
        price = browser.find_elements_by_xpath(path)[0].text

    #SKU
    path = '//*[@class="article-number text--small color-gray-light"]'
    sku = browser.find_elements_by_xpath(path)[0].text

    # Title
    path = '//*[@class="h3 headline"]'
    title = browser.find_elements_by_xpath(path)[0].text

    # Try to show more data
    try:
        path = '//*[@class="icon nm-icon nm-icon-arrow-down"]'
        browser.find_elements_by_xpath(path)[0].click()
    except:
        pass

    # Getting Main Data
    path = '//*[@class="prd-features__feature-list js-prd-features__feature-list prd-features__feature-list--visible"]//dd'
    values = [el.text for el in browser.find_elements_by_xpath(path)]

    path = '//*[@class="prd-features__feature-list js-prd-features__feature-list prd-features__feature-list--visible"]//dt'
    cols = [el.text for el in browser.find_elements_by_xpath(path)]
    main_data = pd.DataFrame([values], columns=cols)

    item_df = pd.DataFrame({'SKU':sku,
                            'Title':title,
                            'Raw Dims':raw_dims,
                            'Price':price,
                            'URL':browser.current_url}, index=[0])

    item_df = pd.concat([item_df, main_data], axis=1)
    product_df = pd.concat([product_df,item_df])
    sleep(1)

    if p%20==0:
        display(item_df)


    df_no_opt = pd.concat([df_no_opt, product_df], ignore_index=True)
    #df.to_csv('Holzland_Options.csv')

    print('Product size:', product_df.shape[0], '|', 'All products size:', df_no_opt.shape[0])




Product 0


,SKU,Title,Raw Dims,Price,URL,Marke,Länge,Holzart / Dekornachbildung,Breite,Furnierqualität,Stärke,Plattenaufbau (Schichten),Grundierfilm,Verleimung
0,Artikelnummer: 7001316447,Elka Naturholzplatte vita Fichte 3-schichtig -...,19 x 1025 x 2525 mm,"61,47 € / Stk.",https://www.holzland.de/p/naturholzplatte-vita...,Elka,2525 mm,Fichte,1025 mm,B/C,19 mm,3,nein,AW 100


Product size: 1 | All products size: 1
Product 1
Product size: 1 | All products size: 2
Product 2
Product size: 1 | All products size: 3
Product 3
Product size: 1 | All products size: 4
Product 4
Product size: 1 | All products size: 5
Product 5
Product size: 1 | All products size: 6
Product 6
Product size: 1 | All products size: 7
Product 7
Product size: 1 | All products size: 8
Product 8
Product size: 1 | All products size: 9
Product 9
Product size: 1 | All products size: 10
Product 10
Product size: 1 | All products size: 11
Product 11
Product size: 1 | All products size: 12
Product 12
Product size: 1 | All products size: 13
Product 13
Product size: 1 | All products size: 14
Product 14
Product size: 1 | All products size: 15
Product 15
Product size: 1 | All products size: 16
Product 16
Product size: 1 | All products size: 17
Product 17
Product size: 1 | All products size: 18
Product 18
Product size: 1 | All products size: 19
Product 19
Product size: 1 | All products size: 20
Product 2

,SKU,Title,Raw Dims,Price,URL,Marke,Länge,Dekornummer,Breite,Dekorbezeichnung,Stärke,Dekorstruktur
0,Artikelnummer: 7000310768,Westag & Getalit Rückwand-System-Set MFC SL239...,2960 x 585 x 13 mm,"135,80 € / Stk.",https://www.holzland.de/p/rueckwand-system-set...,Westag & Getalit,2960 mm,SL239,585 mm,salome manila,13 mm,Colin (C)


Product size: 1 | All products size: 21
Product 21
Product size: 1 | All products size: 22
Product 22
Product size: 1 | All products size: 23
Product 23
Product size: 1 | All products size: 24
Product 24
Product size: 1 | All products size: 25
Product 25
Product size: 1 | All products size: 26
Product 26
Product size: 1 | All products size: 27
Product 27
Product size: 1 | All products size: 28
Product 28
Product size: 1 | All products size: 29
Product 29
Product size: 1 | All products size: 30
Product 30
Product size: 1 | All products size: 31
Product 31
Product size: 1 | All products size: 32
Product 32
Product size: 1 | All products size: 33
Product 33
Product size: 1 | All products size: 34
Product 34
Product size: 1 | All products size: 35
Product 35
Product size: 1 | All products size: 36
Product 36
Product size: 1 | All products size: 37
Product 37
Product size: 1 | All products size: 38
Product 38
Product size: 1 | All products size: 39
Product 39
Product size: 1 | All products 

...

Product size: 1 | All products size: 821
Product 821
Product size: 1 | All products size: 822
Product 822
Product size: 1 | All products size: 823
Product 823
Product size: 1 | All products size: 824
Product 824
Product size: 1 | All products size: 825
Product 825
Product size: 1 | All products size: 826
Product 826
Product size: 1 | All products size: 827
Product 827
Product size: 1 | All products size: 828
Product 828
Product size: 1 | All products size: 829
Product 829
Product size: 1 | All products size: 830
Product 830
Product size: 1 | All products size: 831
Product 831
Product size: 1 | All products size: 832
Product 832
Product size: 1 | All products size: 833
Product 833
Product size: 1 | All products size: 834
Product 834
Product size: 1 | All products size: 835
Product 835
Product size: 1 | All products size: 836
Product 836
Product size: 1 | All products size: 837
Product 837
Product size: 1 | All products size: 838
Product 838
Product size: 1 | All products size: 839
Produ

,SKU,Title,Raw Dims,Price,URL,Marke,Länge,Materialdetailtyp,Breite,Farbton,Stärke,Farbe
0,Artikelnummer: 7000012299,Brügmann DreamDeck WPC Kunststoffclips 100er-S...,,"27,49 €",https://www.holzland.de/p/dreamdeck-wpc-kunsts...,Brügmann,17 mm,Kunststoff (sonstige),25 mm,dunkel,40 mm,anthrazit


Product size: 1 | All products size: 841
Product 841
Product size: 1 | All products size: 842
Product 842
Product size: 1 | All products size: 843
Product 843
Product size: 1 | All products size: 844
Product 844
Product size: 1 | All products size: 845
Product 845
Product size: 1 | All products size: 846
Product 846
Product size: 1 | All products size: 847
Product 847
Product size: 1 | All products size: 848
Product 848
Product size: 1 | All products size: 849
Product 849
Product size: 1 | All products size: 850
Product 850
Product size: 1 | All products size: 851
Product 851
Product size: 1 | All products size: 852
Product 852
Product size: 1 | All products size: 853
Product 853
Product size: 1 | All products size: 854
Product 854
Product size: 1 | All products size: 855
Product 855
Product size: 1 | All products size: 856
Product 856
Product size: 1 | All products size: 857
Product 857
Product size: 1 | All products size: 858
Product 858
Product size: 1 | All products size: 859
Produ

,SKU,Title,Raw Dims,Price,URL,Marke,Länge,Materialdetailtyp,Breite,Farbe,Stärke
0,Artikelnummer: 7001319488,HQ GartenWelt Stelzlager Pad alukaschiert 8 mm,"195 x 195 mm, 12 Stück / Paket","20,95 €",https://www.holzland.de/p/stelzlager-pad-aluka...,HQ GartenWelt,195 mm,Kunststoff (sonstige),195 mm,Schwarz,8 mm


Product size: 1 | All products size: 861
Product 861
Product size: 1 | All products size: 862
Product 862
Product size: 1 | All products size: 863
Product 863
Product size: 1 | All products size: 864
Product 864
Product size: 1 | All products size: 865
CPU times: user 49.7 s, sys: 4.65 s, total: 54.4 s
Wall time: 41min 36s


In [622]:
three_dims_df = pd.read_csv('Holzland_3D_Options.csv', index_col=[0])

In [623]:
df_proc = pd.concat([df_no_opt, df_one_dim, df, three_dims_df])

In [625]:
df_proc.shape

(22714, 79)

In [627]:
df_proc[~df_proc.duplicated(['SKU','Price'])].shape[0]

6775

In [644]:
df_proc[~df_proc.duplicated(['SKU','URL'])].shape[0]

7254

In [628]:
df_proc[~df_proc.duplicated(['URL','SKU','Price'])].shape[0]

7254

#### Get rid of all SKU duplicated and then join with dupl SKU minus pmzr

In [663]:
# First layer: get rid of dupl Price--URL--SKU
first_test = df_proc[(~df_proc.duplicated(['Price','SKU']))]
first_test[~first_test.duplicated(['SKU'], keep=False)].shape[0], first_test.shape[0]

(6390, 6775)

In [664]:
# Second layer: getting rid of pmzr URLs
pmzr = first_test[(first_test.duplicated(['SKU'], keep=False)) & 
                  (~first_test['URL'].str.contains('pmzr'))]

pmzr[pmzr.duplicated(['SKU'], keep=False)].shape[0], pmzr.shape[0]

(0, 114)

In [665]:
first_test = first_test[~first_test.duplicated(['SKU'], keep=False)]
first_test = pd.concat([first_test, pmzr])
first_test.shape

(6504, 79)

In [668]:
(first_test.isna().sum()/first_test.shape[0]*100).sort_values()

SKU                                    0.000000
Title                                  0.000000
Raw Dims                               0.000000
Price                                  0.000000
URL                                    0.000000
Länge                                  2.659902
Breite                                 3.951415
Stärke                                 3.997540
Marke                                 33.563961
Holzart / Dekornachbildung            48.554736
Dekorbezeichnung                      58.948339
Dekornummer                           64.083641
Dekorstruktur                         73.862239
Farbton                               82.395449
Sortierklasse                         82.779828
Oberflächenstruktur Oben/Unten        84.378844
Materialdetailtyp                     85.947109
Farbe                                 88.068881
Profilbezeichnung                     89.129766
Oberflächenbehandlung Hobelware       89.406519
Oberflächenbehandlung Garten          89

# Processing

In [1317]:
test = first_test.copy()
test.shape

(6504, 79)

In [1146]:
first_test[first_test.duplicated(['SKU'])].shape

(0, 79)

### Images

In [1252]:
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)

In [1089]:
#images_df = pd.DataFrame({'Images':np.nan, 'URL':np.nan}, index=[first_test.index])
images_df.shape

(6504, 2)

In [1253]:
first_test=first_test.reset_index(drop=True)

In [1254]:
#browser.quit()
start=5457
for p, idx in enumerate(first_test.index[start:], start=start):
    
    print('Product', p)
    
    
    url = first_test['URL'].loc[idx]
    browser.get(url)
    sleep(2)
    
    path = '//*[@class = "img-wrapper"]/img'
    img_urls = []
    for img in browser.find_elements_by_xpath(path):
        try:
            img_url = img.get_attribute('data-zoom-image')
            print(img_url)
            img_urls.append(img_url)
        except:
            pass
    
    images_df['URL'].loc[idx]=url
    images_df['Images'].loc[idx]= '; '.join(img_urls)

Product 5457
https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Kantholz_Stapel.jpg
https://media.my-holzland.de/zoom/image/hlzco_holzland_pro_kantholz_8.jpg
https://media.my-holzland.de/zoom/image/hlzco_holzland_pro_kantholz_4.jpg
Product 5458
https://media.my-holzland.de/zoom/image/hlzco_Glattkantbrett_Sib_Laerche_einzel.jpg
https://media.my-holzland.de/zoom/image/hlzco_Glattkantbretter_Sib_Laerche_Stapel.jpg
Product 5459
https://media.my-holzland.de/zoom/image/hlzco_Glattkantbrett_Sib_Laerche_einzel.jpg
https://media.my-holzland.de/zoom/image/hlzco_Glattkantbretter_Sib_Laerche_Stapel.jpg
Product 5460
https://media.my-holzland.de/zoom/image/hlzco_Glattkantbrett_Sib_Laerche_einzel.jpg
https://media.my-holzland.de/zoom/image/hlzco_Glattkantbretter_Sib_Laerche_Stapel.jpg
Product 5461
https://media.my-holzland.de/zoom/image/hlzco_Glattkantbrett_Sib_Laerche_einzel.jpg
https://media.my-holzland.de/zoom/image/hlzco_Glattkantbretter_Sib_Laerche_Stapel.jpg
Product 5462
https:/

https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Kantholz_Stapel.jpg
https://media.my-holzland.de/zoom/image/hlzco_holzland_pro_kantholz_8.jpg
https://media.my-holzland.de/zoom/image/hlzco_holzland_pro_kantholz_4.jpg
Product 5496
https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Kantholz_Stapel.jpg
https://media.my-holzland.de/zoom/image/hlzco_holzland_pro_kantholz_8.jpg
https://media.my-holzland.de/zoom/image/hlzco_holzland_pro_kantholz_4.jpg
Product 5497
https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Kantholz_Stapel.jpg
https://media.my-holzland.de/zoom/image/hlzco_holzland_pro_kantholz_8.jpg
https://media.my-holzland.de/zoom/image/hlzco_holzland_pro_kantholz_4.jpg
Product 5498
https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Kantholz_Stapel.jpg
https://media.my-holzland.de/zoom/image/hlzco_holzland_pro_kantholz_8.jpg
https://media.my-holzland.de/zoom/image/hlzco_holzland_pro_kantholz_4.jpg
Product 5499
https://

https://media.my-holzland.de/zoom/image/hlzco_Bretter_frisch.jpg
Product 5538
https://media.my-holzland.de/zoom/image/hlzco_Bretter_frisch.jpg
Product 5539
https://media.my-holzland.de/zoom/image/hlzco_Bretter_frisch.jpg
Product 5540
https://media.my-holzland.de/zoom/image/hlzco_Bretter_frisch.jpg
Product 5541
https://media.my-holzland.de/zoom/image/hlzco_Bretter_frisch.jpg
Product 5542
https://media.my-holzland.de/zoom/image/hlzco_Bretter_frisch.jpg
Product 5543
https://media.my-holzland.de/zoom/image/hlzco_Bretter_frisch.jpg
Product 5544
https://media.my-holzland.de/zoom/image/hlzco_Bretter_frisch.jpg
Product 5545
https://media.my-holzland.de/zoom/image/hlzco_Bretter_frisch.jpg
Product 5546
https://media.my-holzland.de/zoom/image/hlzco_Bretter_frisch.jpg
Product 5547
https://media.my-holzland.de/zoom/image/hlzco_Bretter_frisch.jpg
Product 5548
https://media.my-holzland.de/zoom/image/hlzco_Bretter_frisch.jpg
Product 5549
https://media.my-holzland.de/zoom/image/hlzco_Bretter_frisch.jpg

https://media.my-holzland.de/zoom/image/hlzco_Brett_getrocknet.jpg
Product 5636
https://media.my-holzland.de/zoom/image/hlzco_Brett_getrocknet.jpg
Product 5637
https://media.my-holzland.de/zoom/image/hlzco_Brett_getrocknet.jpg
Product 5638
https://media.my-holzland.de/zoom/image/hlzco_Brett_getrocknet.jpg
Product 5639
https://media.my-holzland.de/zoom/image/hlzco_Brett_getrocknet.jpg
Product 5640
https://media.my-holzland.de/zoom/image/hlzco_Brett_getrocknet.jpg
Product 5641
https://media.my-holzland.de/zoom/image/hlzco_Brett_getrocknet.jpg
Product 5642
https://media.my-holzland.de/zoom/image/hlzco_Brett_getrocknet.jpg
Product 5643
https://media.my-holzland.de/zoom/image/hlzco_Brett_getrocknet.jpg
Product 5644
https://media.my-holzland.de/zoom/image/hlzco_Brett_getrocknet.jpg
Product 5645
https://media.my-holzland.de/zoom/image/hlzco_Brett_getrocknet.jpg
Product 5646
https://media.my-holzland.de/zoom/image/hlzco_Brett_getrocknet.jpg
Product 5647
https://media.my-holzland.de/zoom/image/

https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Dielen_Fichte_saegerauh.jpg
https://media.my-holzland.de/zoom/image/hlzco_Dielen_1.jpg
https://media.my-holzland.de/zoom/image/hlzco_Dielen_2.jpg
Product 5709
https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Dielen_Fichte_saegerauh.jpg
https://media.my-holzland.de/zoom/image/hlzco_Dielen_1.jpg
https://media.my-holzland.de/zoom/image/hlzco_Dielen_2.jpg
Product 5710
https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Dielen_Fichte_saegerauh.jpg
https://media.my-holzland.de/zoom/image/hlzco_Dielen_1.jpg
https://media.my-holzland.de/zoom/image/hlzco_Dielen_2.jpg
Product 5711
https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Dielen_Fichte_saegerauh.jpg
https://media.my-holzland.de/zoom/image/hlzco_Dielen_1.jpg
https://media.my-holzland.de/zoom/image/hlzco_Dielen_2.jpg
Product 5712
https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Dielen_Fichte_saegerauh.jpg
h

...

https://media.my-holzland.de/zoom/image/70653__PICEA_RPR_00_-18_5x146_mm_US2.jpg
https://media.my-holzland.de/zoom/image/70653_M-8168_Picea_RPR_00_18_5x146__Nord_Fichte.jpg
https://media.my-holzland.de/zoom/image/70653_navb6.jpg
Product 6257
https://media.my-holzland.de/zoom/image/70653_M-8167_Picea_RPR_00_18_5x121__Nord_Fichte.jpg
https://media.my-holzland.de/zoom/image/70653_navb6.jpg
Product 6258
https://media.my-holzland.de/zoom/image/70653__PICEA_RPR_00_-18_5x146_mm_US2.jpg
https://media.my-holzland.de/zoom/image/70653_M-8168_Picea_RPR_00_18_5x146__Nord_Fichte.jpg
https://media.my-holzland.de/zoom/image/70653_navb6.jpg
Product 6259
https://media.my-holzland.de/zoom/image/70653__PICEA_RPR_00_-18_5x146_mm_US2.jpg
https://media.my-holzland.de/zoom/image/70653_M-8168_Picea_RPR_00_18_5x146__Nord_Fichte.jpg
https://media.my-holzland.de/zoom/image/70653_navb6.jpg
Product 6260
https://media.my-holzland.de/zoom/image/70653_M-8167_Picea_RPR_00_18_5x121__Nord_Fichte.jpg
https://media.my-holz

https://media.my-holzland.de/zoom/image/hlzco_Terrassendiele_Douglasie_genutet_geriffelt.jpg
https://media.my-holzland.de/zoom/image/hlzco_Terrassendiele_Douglasie_geriffelt_genutet.jpg
Product 6299
https://media.my-holzland.de/zoom/image/hlzco_Terrassendiele_Douglasie_genutet_geriffelt.jpg
https://media.my-holzland.de/zoom/image/hlzco_Terrassendiele_Douglasie_geriffelt_genutet.jpg
Product 6300
https://media.my-holzland.de/zoom/image/hlzco_Terrassendiele_Douglasie_genutet_geriffelt.jpg
https://media.my-holzland.de/zoom/image/hlzco_Terrassendiele_Douglasie_geriffelt_genutet.jpg
Product 6301
https://media.my-holzland.de/zoom/image/hlzco_Terrassendiele_Douglasie_genutet_geriffelt.jpg
https://media.my-holzland.de/zoom/image/hlzco_Terrassendiele_Douglasie_geriffelt_genutet.jpg
Product 6302
https://media.my-holzland.de/zoom/image/hlzco_Terrassendiele_Douglasie_genutet_geriffelt.jpg
https://media.my-holzland.de/zoom/image/hlzco_Terrassendiele_Douglasie_geriffelt_genutet.jpg
Product 6303
https

https://media.my-holzland.de/zoom/image/hlzco_Garten_Terrassendiele_Sib_Laerche_genutet_geriffelt.jpg
https://media.my-holzland.de/zoom/image/hlzco_Garten_Terrassendiele_Sib_Laerche_geriffelt_genutet.jpg
Product 6362
https://media.my-holzland.de/zoom/image/hlzco_Garten_Terrassendiele_Sib_Laerche_beidseitig_glatt.jpg
Product 6363
https://media.my-holzland.de/zoom/image/hlzco_Garten_Terrassendiele_Sib_Laerche_beidseitig_glatt.jpg
Product 6364
https://media.my-holzland.de/zoom/image/hlzco_Garten_Terrassendiele_Sib_Laerche_beidseitig_glatt.jpg
Product 6365
https://media.my-holzland.de/zoom/image/hlzco_Garten_Terrassendiele_Sib_Laerche_beidseitig_glatt.jpg
Product 6366
https://media.my-holzland.de/zoom/image/hlzco_Garten_Terrassendiele_Sib_Laerche_beidseitig_glatt.jpg
Product 6367
https://media.my-holzland.de/zoom/image/hlzco_Terrassendiele_Douglasie_beidseitig_franz_Profil.jpg
Product 6368
https://media.my-holzland.de/zoom/image/hlzco_Terrassendiele_Douglasie_beidseitig_franz_Profil.jpg
Pr

https://media.my-holzland.de/zoom/image/hlzco_Leimholzplatte_Kiefer_1.jpg
https://media.my-holzland.de/zoom/image/hlzco_Leimholzplatte_Kiefer_2.jpg
Product 6420
https://media.my-holzland.de/zoom/image/hlzco_Leimholzplatte_Kiefer_1.jpg
https://media.my-holzland.de/zoom/image/hlzco_Leimholzplatte_Kiefer_2.jpg
Product 6421
https://media.my-holzland.de/zoom/image/hlzco_Holzwerkstoffe_Tischlerplatte_Eiche_keilgezinkt.jpg
Product 6422
https://media.my-holzland.de/zoom/image/hlzco_Rauhspund.jpg
Product 6423
https://media.my-holzland.de/zoom/image/hlzco_Rauhspund.jpg
Product 6424
https://media.my-holzland.de/zoom/image/hlzco_Rauhspund.jpg
Product 6425
https://media.my-holzland.de/zoom/image/hlzco_Rauhspund.jpg
Product 6426
https://media.my-holzland.de/zoom/image/hlzco_Rauhspund.jpg
Product 6427
https://media.my-holzland.de/zoom/image/hlzco_Rauhspund.jpg
Product 6428
https://media.my-holzland.de/zoom/image/hlzco_Rauhspund.jpg
Product 6429
https://media.my-holzland.de/zoom/image/hlzco_Massivholz

https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Balkonbretter_Fichte.jpg
Product 6479
https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Latten_roh_impr_gruen.jpg
Product 6480
https://media.my-holzland.de/zoom/image/hlzco_Quadratleisten_Kiefer.jpg
Product 6481
https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Rundkantenprofil_Fichte_einzel.jpg
Product 6482
https://media.my-holzland.de/zoom/image/hlzco_Rundstaebe_Kiefer.jpg
Product 6483
https://media.my-holzland.de/zoom/image/hlzco_Rundstaebe_Kiefer.jpg
Product 6484
https://media.my-holzland.de/zoom/image/hlzco_Rundstaebe_Kiefer.jpg
Product 6485
https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Latten_roh_01.jpg
https://media.my-holzland.de/zoom/image/hlzco_holzland_pro_latten_1_AUS.jpg
https://media.my-holzland.de/zoom/image/hlzco_holzland_pro_Latten_191.jpg
Product 6486
https://media.my-holzland.de/zoom/image/hlzco_Massivholz_Hobelware_Latten_roh_01.jpg
https://media

### Dimentions

In [1318]:
test['Length'] = test['Raw Dims'].str.split(' x ', expand=True)[0]
test['Width'] = test['Raw Dims'].str.split(' x ', expand=True)[1]
test['Thickness'] = test['Raw Dims'].str.split(' x ', expand=True)[2]

In [1319]:
test[['Length','Width','Thickness','Länge','Breite','Stärke']].sample()

,Length,Width,Thickness,Länge,Breite,Stärke
4643,24,200,5000 mm,5000 mm,200 mm,24 mm


In [1320]:
test[test['Stärke'].fillna('').str.contains(r'\.')]['Stärke'].shape#sample(10)

(740,)

In [1321]:
test['Length, mm'] = test['Länge'].str.replace(r'[^\d\.]','').astype(float)
test['Width, mm'] = test['Breite'].str.replace(r'[^\d\.]','').astype(float)
test['Thickness, mm'] = test['Stärke'].str.replace(r'[^\d\.]','').astype(float)

In [1322]:
test['Thickness, mm'].sort_values(ascending=False).head(5)

4398    440.0
4705    400.0
4441    400.0
4397    400.0
4387    400.0
Name: Thickness, mm, dtype: float64

In [1323]:
test = test[(~test['Length, mm'].isna()) & 
             (~test['Width, mm'].isna()) & 
             (~test['Thickness, mm'].isna())]
test = test.reset_index(drop=True)
test.shape

(6232, 85)

### Keywords

In [1561]:
from df2gspread import gspread2df as g2d
from oauth2client.service_account import ServiceAccountCredentials
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Desktop/UpWork/gs-credentials.json', scope) 
keywords = g2d.download(gfile=spreadsheet_key, wks_name = 'Keywords', credentials=credentials, col_names=True, row_names=True)

In [1562]:
rename_cols = {'Produktart':'Product Type', 
               'Holztyp':'Wood Type',
               'Qualität':'Quality',
               'Oberflächenqualität':'Surface Quality',
               'Sortierklasse':'Sorting Class',
               'Trocknung':'Drying Method'}
keywords = keywords.rename(columns=rename_cols)

In [1563]:
for col in keywords.columns:
    keywords[col] = keywords[col].apply(lambda x: np.nan if x == '' or x == ' ' or x== '-777' else x)

# Combine all columns to mine text form

In [1326]:
test['Combined'] = ''
for col in test.columns:
    test['Combined'] = test['Combined'] + '; ' + test[col].astype(str)

In [1327]:
test['Combined'] = test['Combined'].str.replace(r'\; nan','').str.replace(r'^; ','')

###  Quality

In [1481]:
test['Quality'] = ''
for i in test.index:
    test['Quality'].loc[i] = []
    
    for qual in keywords['Quality'].unique():
        qual = str(qual) 
        pat = r' {} |^{} | {}$| {}\;'.format(qual.lower(), qual.lower(), qual.lower(), qual.lower())
        
        if qual!= 'nan' and len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and qual.lower() not in ', '.join(test['Quality'].loc[i]):
            test['Quality'].loc[i].append(qual)
            
    try:
        test['Quality'].loc[i] = ', '.join(test['Quality'].loc[i])
    except:
        pass

test['Quality'] =  test['Quality'].apply(lambda x: np.nan if x==' ' or x=='' else x )
test['Quality'] = test['Quality'].fillna(test['Furnierqualität']).fillna(test['Sortierklasse'])
test['Quality'] = test['Quality'].fillna(test['Furnierqualität']).fillna(test['Schnittklasse'])

test['Quality'].value_counts()

U/S hobelfallend     704
A-Sortierung         223
A/B                  155
GK I/III             141
B/C                   89
Sexta und besser      78
B-Sortierung          75
GK II/III             52
A/S                   51
GK I/II               40
BB/CC                 28
B/BB                  27
Quinta und besser     23
BB/BB                 16
C/C                   12
GK III                10
II/III                 8
II/II                  8
B/C+                   8
C+/C                   7
S/BB                   6
BB/WG                  5
GK I/IV                4
A                      1
Altholz                1
Name: Quality, dtype: int64

### Sorting Class

In [1330]:
sort_classes = ['S7', 'S10', 'S13', 'C16M', 'C24M', 'C30M', 'D30', 'D35', 'D40',
               'D60', 'D70', 'geringe tragfähigkeit', 'mittlere tragfähigkeit',
               'hohe tragfähigkeit']

In [1331]:
test['Sorting Class'] = ''
for i in test.index:
    test['Sorting Class'].loc[i] = []
    for sort_class in sort_classes:
        pat = r' {} |^{} | {}$| {}\;'.format(sort_class.lower(), sort_class.lower(), sort_class.lower(), sort_class.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and sort_class.lower() not in ', '.join(test['Sorting Class'].loc[i]):
            test['Sorting Class'].loc[i].append(sort_class)
       
    try:
        test['Sorting Class'].loc[i] = ', '.join(test['Sorting Class'].loc[i])
    except:
        pass
test['Sorting Class'] =  test['Sorting Class'].apply(lambda x: np.nan if x==' ' or x=='' else x )
test['Sorting Class'] =  test['Sorting Class'].fillna(test['Sortierklasse nach DIN 4074']).fillna(test['Festigkeitsklasse nach EN 338'])
test['Sorting Class'].value_counts()

C 24    92
S7      40
S10      9
Name: Sorting Class, dtype: int64

## Main parameters

### Wood Types

In [1332]:
woods = ['Ahorn', 'Tanne', 'Sperrholz', 'Nussbaum', 'Erle',
        'Bongossi (Azobé)', 'Birke', 'Buche', 'Douglasie',
        'Edelkastanie', 'Eiche', 'Elsbeere', 'Erle', 'Esche', 'Fichte',
        'Kiefer', 'Kirschbaum', 'Lärche', 'Linde', 'Mahagoni', 'Teak',
        'Pappel', 'Robinie', 'Tanne', 'Ulme', 'Walnuss', 'Weide', 'Birne',
        'Fichte/Tanne', 'Fichte/Kiefer']

In [1333]:
test['Meta Wood'] = ''
for i in test.index:
    for wood in woods:
        pat = r' {} |^{} | {}$'.format(wood.lower(), wood.lower(), wood.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0:
            test['Meta Wood'].loc[i] = wood

In [1334]:
test['Meta Wood'] = test['Meta Wood'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [1335]:
test['Wood Type'] = test['Holzart / Dekornachbildung'].fillna(test['Meta Wood'])
test['Wood Type'] = test['Wood Type'].fillna(test['Trägermaterial'])
test['Wood Type'] = test['Wood Type'].fillna(test['Materialdetailtyp'])

#### Cheking wrong typs and woods

In [1336]:
wrong_wood = ['Holzwerkstoff (MDF)', 'Glas', 'Aluminium', 'Metall', 'Granit', 'Polymere', 'Granito',
              'Kunststoff','Laminiert','Gipskarton, Furniert','PVC','Beschichtet', 'Melaminbeschichtet' ,
              'Furniert','PVC-freie Werkstoffmischung',  'Foliert',  'Laminiert', 'Polymere','Polycarbonat',
              'Polyvinylchlorid (PVC)','Gipskarton','German Compact Composite (GCC)','Bandstahl','Foliert',
              'Faseprofil','Metall, Kunststoff', 'Textil, Kunststoff','Rotbuche','Multiplex',
              'Qualitätsspanplatte mit HPL-Beschichtung Profil aus Hart-PVC','Holz-Kunststoff-Verbundwerkstoffe (WPC)'
              ,'Beschichtete Flachpressplatte', 'Hart-PVC-Leiste','Lackiert','WPC', 
              'Weich-PVC','Unbehandelt','Kunststoffbeschichtet','Kunststoffummantelter Hartfaserkern',
               'Stahl','Beschichtete Flachpressplatte, Hart-PVC-Leiste','Beidseitig mit HPL beschichtete Spanplatte',
              'Polycarbonat', 'German Compact Composite (GCC)', 
              'Foliert', 'WPC', 'Multiplex', 'Beschichtet', 'Stahl','Kunststoff (sonstige)','Verbundwerkstoff (sonstige)']

In [1337]:
# Percentage of wrong wood types
test[(test['Wood Type'].isin(wrong_wood))].shape[0]/test.shape[0]*100

0.7060333761232349

In [1338]:
test[(test['Wood Type'].isin(wrong_wood))]['Wood Type'].value_counts()

Kunststoff (sonstige)          18
Aluminium                      15
Verbundwerkstoff (sonstige)    11
Name: Wood Type, dtype: int64

In [1339]:
test = test[(~test['Wood Type'].isin(wrong_wood))]
test.shape

(6188, 90)

In [1341]:
test['Wood Type'].value_counts()

Fichte                                  1820
WPC (Wood Plastic Composite)             397
MDF                                      328
Lärche                                   298
Eiche                                    249
Buche                                    160
Kiefer                                   150
Esche                                    148
Bangkirai                                123
Douglasie                                 96
BPC (Bamboo Plastic Composite)            75
Ahorn                                     70
Pinie                                     63
Garapa                                    62
Kiefer / Fichte                           60
Meranti                                   45
Nussbaum                                  43
Birke                                     36
HDF                                       31
Teak                                      25
Guyana Teak                               23
GCC (German Compact Composite)            23
Ipe       

### Surface

In [1342]:
strong_surfaces = ['geschliffen', 'gehobelt', 'gefast',
                   'sägerau', 'geriffelt', 'glatt gehobelt', 'glatte', 'gefräst',
                   'gebürstet', 'strukturiert', '1-Seitig gehobelt',
                   '2-Seitig gehobelt', '3-Seitig gehobelt', '4-Seitig gehobelt',
                   'genutet', 'unbehandelt', 'naturbelassen', 
                   'deckend', 'lasiert', 'vorgeölt', 'kesseldruckimprägniert (KDI)',
                   'hitzebehandelt', 'thermobehandelt', 'wachsen', 'imprägniert',
                   'Scharfkantig']

weak_surfaces = ['roh', 'keine']

In [1343]:
test['Combined Surface Treat'] = ''
for i in test.index:
    test['Combined Surface Treat'].loc[i] = []
    for surface in weak_surfaces:
        pat = r' {} |^{} | {}$'.format(surface.lower(), surface.lower(), surface.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and surface.lower() not in ', '.join(test['Combined Surface Treat'].loc[i]):
            test['Combined Surface Treat'].loc[i].append(surface)
    
    for surface in strong_surfaces:
        if surface.lower() in test['Combined'].loc[i].lower() and surface.lower() not in ', '.join(test['Combined Surface Treat'].loc[i]):
            test['Combined Surface Treat'].loc[i].append(surface)
    try:
        test['Combined Surface Treat'].loc[i] = ', '.join(test['Combined Surface Treat'].loc[i])
    except:
        pass

In [1344]:
test['Combined Surface Treat'] = test['Combined Surface Treat'].apply(lambda x: np.nan if x==' ' or x=='' else x )
test['Surface Treatment'] = test['Combined Surface Treat']

In [1345]:
cols = ['Oberflächenbehandlung Bauholz', 'Oberflächenbehandlung Garten',
        'Oberflächenstruktur Oben/Unten','Oberflächenbehandlung Hobelware']
for col in cols:
    test['Surface Treatment'] = test['Surface Treatment'].fillna(test[col])
test['Surface Treatment'] = test['Surface Treatment'].str.lower()
#test['Surface Treatment'].value_counts()

### Surface Quality

In [1346]:
surf_quality = ['NSI', 'SI']

In [1347]:
test['Surface Quality'] = ''
for i in test.index:
    test['Surface Quality'].loc[i] = []
    for surf_qual in surf_quality:
        pat = r' {} |^{} | {}$|[a-z] {}\;'.format(surf_qual.lower(), surf_qual.lower(), surf_qual.lower(), surf_qual.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and surf_qual.lower() not in ', '.join(test['Surface Quality'].loc[i]):
            test['Surface Quality'].loc[i].append(surf_qual)
            
    try:
        test['Surface Quality'].loc[i] = ', '.join(test['Surface Quality'].loc[i])
    except:
        pass
test['Surface Quality'] =  test['Surface Quality'].apply(lambda x: np.nan if x==' ' or x=='' else x )
test['Surface Quality'] =  test['Surface Quality'].fillna(test['Oberflächenqualität'])

In [1349]:
test['Oberflächenqualität'].value_counts()

Si       189
Nsi      152
Name: Oberflächenqualität, dtype: int64

### Drying method

In [1498]:
test['Combined Drying Method'] = ''
for i in test.index:
    test['Combined Drying Method'].loc[i] = []
    for method in keywords['Drying Method'].unique():
        method = str(method) 
        
        pat = r' {} |^{} | {}$| {}\;'.format(method.lower(), method.lower(), method.lower(), method.lower())
        if method!='nan' and len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and method.lower() not in ', '.join(test['Combined Drying Method'].loc[i]):
            test['Combined Drying Method'].loc[i].append(method)

    try:
        test['Combined Drying Method'].loc[i] = ', '.join(test['Combined Drying Method'].loc[i])
    except:
        pass

In [1499]:
test['Combined Drying Method'] =  test['Combined Drying Method'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [1500]:
test['Drying Method'] = test['Combined Drying Method']

In [1501]:
test['Drying Method'].value_counts()

frisch    173
AD          3
Name: Drying Method, dtype: int64

In [1512]:
test[test['Title'].str.contains(' frisch ')]['Title'].shape

(170,)

In [1511]:
test[test['Combined Drying Method']=='AD'].shape#

(3, 98)

### Updating Product types

In [1564]:
test['Product Type'] = np.nan
test['Product Type'].loc[0] = ''

In [1565]:
for i in test.index:
    test['Product Type'].loc[i] = []
    for key_word in keywords['Product Type'].unique():
        if key_word.lower() in test['Combined'].loc[i].lower():
            if any([key_word.lower() in x.lower() for x in test['Product Type'].loc[i]]):
                continue
            
            else:
                test['Product Type'].loc[i].append(key_word)
    test['Product Type'].loc[i] = ', '.join(test['Product Type'].loc[i])

In [1566]:
wrong_products = ['Befestigungssysteme', 'Zubehör','Holzschraube','Terrassenbauschraube', 'Terrassenschraube',
                 'Terrassenschraube']

In [1567]:
test[(~test['Product Type'].isin(wrong_products)) &
           (~test['Title'].str.contains(r'schraube'))].shape

(5221, 98)

In [1568]:
test = test[(~test['Product Type'].isin(wrong_products)) &
            (~test['Title'].str.contains(r'schraube'))]
test.shape

(5221, 98)

In [1569]:
test['Product Type'].value_counts()

Arbeitsplatte            1423
Terrassendiele            985
Brett                     482
Latte                     347
Glattkantbrett            277
Profilholz                238
Brettschichtholz          211
Latte, Leimholz           152
Fassadenprofil            125
Kantholz                   96
Bohle                      96
Rauspund                   89
Konstruktionsholz          84
Konstruktionsvollholz      83
Sperrholz                  72
                           64
Rahmen, Rahmenholz         60
Rahmen                     55
Balken                     47
Multiplexplatte            33
Rechteckleiste             32
Latte, Spanplatte          29
Fensterladen               25
Unterkonstruktion          22
Arbeitsplatte, Bohle       21
Rhombus                    13
Latte, Terrassendiele      12
Quadratleiste               9
Rundstab                    8
Winkelleiste                6
Konstruktionsbohle          6
Schalbrett                  5
Inneneckleiste              4
Außeneckle

## Checking Products

In [1528]:
test[test['Product Type'].isna()].shape#['Decklage'].value_counts()#nunique()

(0, 98)

In [1529]:
test = test[(test['Product Type']!='') | (test['Dekorbezeichnung'].isna())]
test.shape

(5221, 98)

In [1530]:
test = test[(~test['Product Type'].isna()) | (test['Dekorbezeichnung'].isna())]
test.shape

(5221, 98)

In [1532]:
test[(test['Product Type']=='')].shape

(64, 98)

### Price

In [1372]:
test[test['Price'].str.contains(r'(?<=\d)\.(?=\d+\.)')].shape

(0, 96)

In [1373]:
test['Price'] = test['Price'].str.replace('\,','.')
test['Price'] = test['Price'].str.replace(r'[^\.\d]','')
test['Price'] = test['Price'].str.replace('\.$','').str.replace('\.00$','')
test['Price'] = test['Price'].str.replace(r'(?<=\d)\.(?=\d+\.)','')
test['Price'] = test['Price'].astype(float)

### SKU

In [1374]:
test['SKU'] = test['SKU'].str.replace(r'[^\d]','')

In [1375]:
test['SKU'].sample()

3310    7000314773
Name: SKU, dtype: object

### Matching keywords

In [1376]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [1377]:
main_cols = ['URL','SKU','Product Type','Wood Type','Surface Treatment',
             'Drying Method','Sorting Class','Quality','Combined', 'Surface Quality']

In [1378]:
test['Matched Keywords'] = np.nan
test['Other Keywords'] = np.nan
test = test.fillna('-777')
keywords = keywords.fillna('-999')

In [1379]:
%%time
def get_keywords(row):
    row['Matched Keywords'] = {}
    row['Other Keywords'] = {}
    
    matched_cols=[]
    for col in ['Product Type','Wood Type','Surface Treatment','Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        for key_word in keywords[col].unique():
            if fuzz.token_set_ratio(str(row[col]).lower(), key_word.lower()) > 90 or key_word.lower() in str(row[col]):
                row['Matched Keywords'][row[col]] = key_word
                matched_cols.append(col)
    
    if len(matched_cols)==0:
        row['Other Keywords'] = np.nan
        row['Matched Keywords'] = np.nan
    
    for col in ['Product Type','Wood Type','Surface Treatment','Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        if col not in matched_cols and len(matched_cols)>0 and row[col]!=-999:
            row['Other Keywords'][col] = row[col]
    
    return row
test = test.apply(get_keywords, axis=1)

CPU times: user 1min 38s, sys: 924 ms, total: 1min 39s
Wall time: 2min 19s


In [1380]:
test[['Matched Keywords','Other Keywords']].isna().sum()/test.shape[0]*100

Matched Keywords    1.39526
Other Keywords      1.39526
dtype: float64

### Cleaning NaNs



In [1381]:
for col in test.columns:
    test[col] =  test[col].apply(lambda x: np.nan if x =='' or x == 'nan' or x == ' ' or x == '-777' or x == '-999' else x)

In [1382]:
test.shape

(5232, 98)

In [1383]:
test[(test['Matched Keywords'].isna())].shape

(73, 98)

In [1384]:
test[(test['Matched Keywords'].isna()) & 
          (test['Wood Type']=='Holz')].shape

(2, 98)

In [1385]:
test[(~test['Matched Keywords'].isna()) #&
          ]['Product Type'].nunique()#value_counts()

25

### Pre-cleaning

In [1386]:
for col in test.columns:
    test[col] = test[col].apply(lambda x: np.nan if x == '' or x == ' ' or x=='-999' or x=='-777' or x=='nan'else x)

# GS

In [1570]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Desktop/UpWork/gs-credentials.json', scope) 
gc = gspread.authorize(credentials)
spreadsheet_key = '1LVxutY3DilXH6t3P0dI-oYl3bA9nWcmwUva7CukV98c'

In [1571]:
sheets_clean = test.copy()

In [1573]:
sheets_clean = test[~test['Matched Keywords'].isna()]
sheets_clean.shape

(5159, 98)

In [1574]:
sheets_clean[(sheets_clean['Product Type'].isna())]['Title'].shape

(0,)

In [1575]:
sheets_clean['Website Name'] = 'www.holzland.de'

In [1576]:
sheets_clean=pd.merge(sheets_clean, images_df.drop_duplicates('URL'), on='URL', how='left')
sheets_clean['Images'].isna().sum(), sheets_clean.shape[0]

(0, 5159)

In [1577]:
ren_cols = {'Title':'Product Title',
            'Width, mm':'Width (mm)',
            'Length, mm':'Length (mm)', 
            'Thickness, mm':'Thickness (mm)',
            'Price':'Price (Euro)'}
sheets_clean = sheets_clean.rename(columns = ren_cols)

cols = ['Website Name', 'URL','Product Title','Product Type', 'Wood Type', 'Width (mm)', 'Length (mm)', 'Thickness (mm)', 
        'Price (Euro)','Quality', 'Surface Treatment','Drying Method', 'Surface Quality',
        'Sorting Class','SKU','Images']

sheets_clean = sheets_clean[cols].rename(columns = ren_cols)

In [1578]:
sheets_clean = sheets_clean[~sheets_clean['Product Type'].isin(wrong_products)]
sheets_clean.shape

(5159, 16)

In [1579]:
sheets_clean[(sheets_clean['Width (mm)'].isna()) &
             (sheets_clean['Length (mm)'].isna()) &
             (sheets_clean['Thickness (mm)'].isna())].shape[0]/sheets_clean.shape[0]*100

0.0

In [1580]:
d2g.upload(sheets_clean.fillna('-'),
           spreadsheet_key,
           'Holzland',
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=True)

<Worksheet 'Holzland' id:911635684>